In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

from matplotlib.colors import LogNorm
from scipy.integrate import odeint
sns.set_context("talk", font_scale=1.4)


# for using LaTeX fonts
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})
# for Palatino and other serif fonts use:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

### Get order with pathpy
 

In [2]:
import pathpy as pp


In [3]:
toy_paths = pp.Paths()
toy_paths.add_path('a,c,d', 2)
toy_paths.add_path('b,c,e', 2)
print(toy_paths)

Total path count: 		4.0 
[Unique / Sub paths / Total]: 	[2.0 / 20.0 / 24.0]
Nodes:				5 
Edges:				4
Max. path length:		2
Avg path length:		2.0 
Paths of length k = 0		0.0 [ 0.0 / 12.0 / 12.0 ]
Paths of length k = 1		0.0 [ 0.0 / 8.0 / 8.0 ]
Paths of length k = 2		4.0 [ 2.0 / 0.0 / 4.0 ]



In [6]:
hon_1 = pp.HigherOrderNetwork(toy_paths)
pp.visualisation.plot(hon_1)
print(hon_1.transition_matrix())

print(hon_1.likelihood(toy_paths, log=False))




  (1, 0)	1.0
  (1, 3)	1.0
  (2, 1)	0.5
  (4, 1)	0.5
0.0625


In [7]:
hon_2 = pp.HigherOrderNetwork(toy_paths, k=2)
print(hon_2.transition_matrix())
hon_2.likelihood(toy_paths, log=False)

  (1, 0)	1.0
  (3, 2)	1.0


1.0

In [8]:
from scipy.stats import chi2

d = hon_2.degrees_of_freedom() - hon_1.degrees_of_freedom()
x = - 2 * (hon_1.likelihood(toy_paths, log=True) - hon_2.likelihood(toy_paths, log=True))
p = 1 - chi2.cdf(x, d)

print('The p-value of the null hypothesis (first-order model) is {0}'.format(p))

The p-value of the null hypothesis (first-order model) is 0.018531677751199016


In [9]:
toy_paths *= 10

x = - 2 * (hon_1.likelihood(toy_paths, log=True) - hon_2.likelihood(toy_paths, log=True))
p = 1 - chi2.cdf(x, d)

print('The p-value of the null hypothesis (first-order model) is {0}'.format(p))

The p-value of the null hypothesis (first-order model) is 9.581224702515101e-14


In [10]:
mog = pp.MultiOrderModel(toy_paths, max_order=2)

d = mog.degrees_of_freedom(max_order=2) - mog.degrees_of_freedom(max_order=1)
x = - 2 * (mog.likelihood(toy_paths, log=True, max_order=1) - mog.likelihood(toy_paths, log=True, max_order=2))
p = 1 - chi2.cdf(x, d)

print('p value of null hypothesis that data has maximum order 1 = {0}'.format(p))

2022-08-24 16:15:11 [Severity.INFO]	Generating 0-th order layer ...
2022-08-24 16:15:11 [Severity.INFO]	Generating 1-th order layer ...
2022-08-24 16:15:11 [Severity.INFO]	Generating 2-th order layer ...
2022-08-24 16:15:11 [Severity.INFO]	finished.
p value of null hypothesis that data has maximum order 1 = 9.094947017729282e-13


In [11]:
mog.estimate_order()


2022-08-24 16:15:16 [Severity.INFO]	Likelihood ratio test for K_opt = 2, x = 55.45177444479563
2022-08-24 16:15:16 [Severity.INFO]	Likelihood ratio test, d_1-d_0 = 2
2022-08-24 16:15:16 [Severity.INFO]	Likelihood ratio test, p = 9.094947017729282e-13


2

In [12]:
random_paths = pp.Paths()
random_paths.add_path('a,c,d', 5)
random_paths.add_path('a,c,e', 5)
random_paths.add_path('b,c,e', 5)
random_paths.add_path('b,c,d', 5)

mog = pp.MultiOrderModel(random_paths, max_order=2)
print('Optimal order = ', mog.estimate_order(random_paths))

2022-08-24 16:15:40 [Severity.INFO]	Generating 0-th order layer ...
2022-08-24 16:15:40 [Severity.INFO]	Generating 1-th order layer ...
2022-08-24 16:15:40 [Severity.INFO]	Generating 2-th order layer ...
2022-08-24 16:15:40 [Severity.INFO]	finished.
2022-08-24 16:15:40 [Severity.INFO]	Likelihood ratio test for K_opt = 2, x = -0.0
2022-08-24 16:15:40 [Severity.INFO]	Likelihood ratio test, d_1-d_0 = 2
2022-08-24 16:15:40 [Severity.INFO]	Likelihood ratio test, p = 1.0
Optimal order =  1


## Try it for the real data

In [3]:
# load the Polity5 data from the excel data frame
timeSeries = pd.read_excel('./../data/polity5/p5v2018.xls')
# do some cleaning
timeSeries = timeSeries.dropna(subset=['year', 'country', 'polity2'])

timeSeries = timeSeries[timeSeries["polity2"] != -66]
timeSeries = timeSeries[timeSeries["polity2"] != -88]

In [4]:
def pathFromTimeSeries(timeSeriesIn):

    paths = pp.Paths()
    

     # split the timeseries into one for each country 
    uniqueCountries = timeSeriesIn['country'].unique()
    countryTimeseries =[]


    for country in uniqueCountries:
        sortedPolity2Country = timeSeriesIn[timeSeriesIn['country'] == country].sort_values(by='year')["polity2"].tolist()
        
        #print(tuple(sortedPolity2Country))

        paths.add_path(tuple(sortedPolity2Country))
        
        #break
        #T=len(sortedPolity2Country)# number of data points
    
    return(paths)

In [5]:
polity2Path = pathFromTimeSeries(timeSeries)

In [6]:
print(polity2Path)

Total path count: 		195.0 
[Unique / Sub paths / Total]: 	[194.0 / 1172303.0 / 1172498.0]
Nodes:				21 
Edges:				326
Max. path length:		218
Avg path length:		87.61538461538461 
Paths of length k = 0		0.0 [ 0.0 / 17280.0 / 17280.0 ]
Paths of length k = 1		1.0 [ 1.0 / 17084.0 / 17085.0 ]
Paths of length k = 2		1.0 [ 1.0 / 16889.0 / 16890.0 ]
Paths of length k = 3		1.0 [ 1.0 / 16695.0 / 16696.0 ]
Paths of length k = 4		0.0 [ 0.0 / 16503.0 / 16503.0 ]
Paths of length k = 5		0.0 [ 0.0 / 16311.0 / 16311.0 ]
Paths of length k = 6		0.0 [ 0.0 / 16119.0 / 16119.0 ]
Paths of length k = 7		2.0 [ 2.0 / 15925.0 / 15927.0 ]
Paths of length k = 8		0.0 [ 0.0 / 15735.0 / 15735.0 ]
Paths of length k = 9		0.0 [ 0.0 / 15545.0 / 15545.0 ]
Paths of length k = 10		1.0 [ 1.0 / 15354.0 / 15355.0 ]
Paths of length k = 11		1.0 [ 1.0 / 15164.0 / 15165.0 ]
Paths of length k = 12		2.0 [ 2.0 / 14974.0 / 14976.0 ]
Paths of length k = 13		1.0 [ 1.0 / 14787.0 / 14788.0 ]
Paths of length k = 14		1.0 [ 1.0 / 14601.0 / 14

In [14]:
polity2Path_graph = pp.HigherOrderNetwork(polity2Path)
pp.visualisation.plot(polity2Path_graph)


In [15]:
mog = pp.MultiOrderModel(polity2Path, max_order=7)
print('Optimal order = ', mog.estimate_order(polity2Path))

2022-08-24 18:52:54 [Severity.INFO]	Generating 0-th order layer ...
2022-08-24 18:52:54 [Severity.INFO]	Generating 1-th order layer ...
2022-08-24 18:52:54 [Severity.INFO]	Generating 2-th order layer ...
2022-08-24 18:53:40 [Severity.INFO]	Generating 3-th order layer ...
2022-08-24 18:53:41 [Severity.INFO]	Generating 4-th order layer ...
2022-08-24 18:53:43 [Severity.INFO]	Generating 5-th order layer ...
2022-08-24 18:53:58 [Severity.INFO]	Generating 6-th order layer ...
2022-08-24 18:58:55 [Severity.INFO]	Generating 7-th order layer ...


: 

: 